### **⏱️ Timeouts and Waits in Selenium WebDriver**

#### **🧠 Why Waits Matter?**

Modern websites are dynamic — elements take **time to load**, appear/disappear, or change.

Without proper waits:

- ❌ Script may **fail randomly**

- ❌ You may get **NoSuchElementException**

- ❌ You'll go mad debugging timing issues 😵

So Selenium gives you **different waiting strategies.**

#### **✅ Selenium Wait Types (with Purpose)**

| Type             | Use Case                          | When to Use?                                   |
|------------------|-----------------------------------|------------------------------------------------|
| Implicit Wait    | Global delay while finding elements | When elements take time to appear randomly     |
| Explicit Wait    | Wait for a specific condition      | Waiting for visibility/clickability of button, text, pop-up etc. |
| Fluent Wait      | Advanced explicit wait with polling | Retry strategy for highly dynamic or flaky elements |
| PageLoadTimeout  | Wait for full page to load         | For pages that take time to fully render/load  |
| ScriptTimeout    | Wait for JavaScript to finish      | If using `JavascriptExecutor` in your tests    |


----------

#### **🔹 STEP 1: Implicit Wait**

##### **📘 What is it?**

Implicit Wait tells WebDriver to **wait for a specified time while searching for elements** before throwing `NoSuchElementException`.

> 💡 Think of it like a lazy waiter:

> “I’ll hang around **just in case** the button shows up in the next 10 seconds…”

In [ ]:
driver.manage().timeouts().implicitlyWait(Duration.ofSeconds(10));

📌 Once set, it applies **globally** to all `findElement()` calls — no need to write it again and again.


##### **🧪 Practical Use Case:**

Let’s say you visit a webpage where the **login button** takes a couple of seconds to render after page load.

Instead of writing:

In [ ]:
WebElement loginBtn = driver.findElement(By.id("login")); // might fail if slow

You do:

In [ ]:
driver.manage().timeouts().implicitlyWait(Duration.ofSeconds(10));
WebElement loginBtn = driver.findElement(By.id("login")); // waits up to 10s

🎯 This prevents flaky tests due to slow loading UI elements.

**⚠️ Pro Tip:**

- **Set it only once** at the beginning of the script.

- **Avoid combining Implicit + Explicit Wait** — it may cause unpredictable timeout behavior.

--------------------

#### **🔹 STEP 2: PageLoad Timeout**

🧠 What is PageLoadTimeout?

- It controls how long Selenium waits for the full page to load after using `driver.get(url)`

- If the page takes more than the specified time, Selenium throws a `TimeoutException`

#### **🔧 When to Use?**

| Scenario                                          | Use PageLoadTimeout? |
| ------------------------------------------------- | -------------------- |
| Page has slow network calls                       | ✅ Yes                |
| Third-party scripts taking too long               | ✅ Yes                |
| You want your test to fail fast on broken pages   | ✅ Yes                |
| All elements are already present in DOM instantly | ❌ Not needed         |


#### **✅ Practical Example (Slow Page Load Simulation)**

We'll use the same site but **simulate slowness** by:

- Setting very low timeout (e.g., 2s)

- And opening a normal page that **takes ~3s to load**

In [ ]:
package seleniumDemo1;

import java.time.Duration;

import org.openqa.selenium.WebDriver;
import org.openqa.selenium.chrome.ChromeDriver;

public class PageLoadTimeoutDemo {
    public static void main(String[] args) {
        WebDriver driver = new ChromeDriver();

        // ❗ Set page load timeout to only 2 seconds
        driver.manage().timeouts().pageLoadTimeout(Duration.ofSeconds(2));

        try {
            // 🔗 Try to load a real page that takes ~2-3 seconds
            driver.get("https://the-internet.herokuapp.com/slow");

            System.out.println("✅ Page loaded successfully!");
        } catch (Exception e) {
            System.out.println("❌ Page took too long to load or failed: " + e.getMessage());
        }

        driver.quit();
    }
}

**🧪 What to Expect?**

- If the page loads within **2 seconds**, it’ll say:

In [ ]:
✅ Page loaded successfully!

- If it doesn’t, you’ll get:

In [ ]:
❌ Page took too long to load or failed: timeout...

Try changing the `pageLoadTimeout` to 5–10 seconds to see the difference.

-----------

#### **✅ What is Explicit Wait?**

Explicit Wait lets you **wait for a specific condition** (like element becoming visible, clickable, etc.) before moving forward.

It's more **precise and smarter** than implicit wait.

#### **🔍 Real-Life Analogy:**

> “Don’t keep standing in the lobby for 10 seconds just hoping the pizza arrives (implicit)…

> Instead, wait only until the doorbell rings! (explicit)” 🍕

#### **🧪 Use Case:**

- Wait for a success message to **appear after clicking a button**

- Wait until a **loading spinner disappears**

- Wait for an alert to **pop up**

- Wait until an element is **clickable**

#### **✅ Sample Code – Wait for Message After Clicking “Remove” Button**

You already used this URL:

`https://the-internet.herokuapp.com/dynamic_controls`

Now do it with **Explicit Wait:**

In [ ]:
package seleniumDemo1;

import java.time.Duration;

import org.openqa.selenium.By;
import org.openqa.selenium.WebDriver;
import org.openqa.selenium.WebElement;
import org.openqa.selenium.chrome.ChromeDriver;
import org.openqa.selenium.support.ui.WebDriverWait;
import org.openqa.selenium.support.ui.ExpectedConditions;

public class ExplicitWaitExample {
    public static void main(String[] args) {
        WebDriver driver = new ChromeDriver();

        driver.manage().window().maximize();
        driver.get("https://the-internet.herokuapp.com/dynamic_controls");

        WebElement removeButton = driver.findElement(By.xpath("//form[@id='checkbox-example']//button"));
        removeButton.click();

        // ⏳ Explicit Wait
        WebDriverWait wait = new WebDriverWait(driver, Duration.ofSeconds(10));
        WebElement message = wait.until(ExpectedConditions.visibilityOfElementLocated(By.id("message")));

        System.out.println("✅ Message displayed: " + message.getText());

        driver.quit();
    }
}

#### **🔍 What just happened?**

- Clicked the **Remove** button

- Waited **until message with ID** `message` **becomes visible**

- Then printed the message 💬

#### **🔧 Common Expected Conditions:**

| Condition                        | Use When                      |
| -------------------------------- | ----------------------------- |
| `visibilityOfElementLocated()`   | Waiting for element to appear |
| `elementToBeClickable()`         | Wait before clicking buttons  |
| `alertIsPresent()`               | Waiting for alerts            |
| `invisibilityOfElementLocated()` | Wait for loader/spinner to go |


----------

#### **✅ What is Fluent Wait?**

Fluent Wait is like an **advanced Explicit Wait**.

It not only waits for a condition — it also:

- ⏱️ **Polls** repeatedly (e.g., every 2 seconds)

- ❌ Ignores specific exceptions (like `NoSuchElementException`)

- 🧠 Gives you **fine control** over retrying

#### **🔍 Real-Life Analogy:**

> “You’re waiting for your friend to come online. You check every 2 seconds, but you ignore if he’s offline (no error). You’ll keep checking for 15 seconds before giving up.” 🕵️‍♂️

#### **✅ When to Use Fluent Wait?**

- The element may appear **randomly**

- API or AJAX response times vary

- You want **custom polling intervals**

- You want to **ignore certain exceptions**



#### **🧪 Example: Wait for Message After Clicking Remove (with Fluent Wait)**

In [ ]:
package seleniumDemo1;

import java.time.Duration;
import java.util.NoSuchElementException;

import org.openqa.selenium.By;
import org.openqa.selenium.WebDriver;
import org.openqa.selenium.WebElement;
import org.openqa.selenium.chrome.ChromeDriver;
import org.openqa.selenium.support.ui.FluentWait;
import org.openqa.selenium.support.ui.Wait;
import org.openqa.selenium.support.ui.ExpectedConditions;

public class FluentWaitExample {
    public static void main(String[] args) {
        WebDriver driver = new ChromeDriver();
        driver.get("https://the-internet.herokuapp.com/dynamic_controls");

        WebElement removeButton = driver.findElement(By.xpath("//form[@id='checkbox-example']//button"));
        removeButton.click();

        // 🕰️ Fluent Wait setup
        Wait<WebDriver> fluentWait = new FluentWait<>(driver)
            .withTimeout(Duration.ofSeconds(15))          // Max wait time
            .pollingEvery(Duration.ofSeconds(2))          // Poll every 2 secs
            .ignoring(NoSuchElementException.class);      // Ignore this exception

        // ✅ Waiting for message to be visible
        WebElement message = fluentWait.until(ExpectedConditions.visibilityOfElementLocated(By.id("message")));

        System.out.println("✅ Message with Fluent Wait: " + message.getText());

        driver.quit();
    }
}

#### **🎯 Key Differences – Explicit vs Fluent**

| Feature            | Explicit Wait   | Fluent Wait            |
| ------------------ | --------------- | ---------------------- |
| Polling control    | ❌ Not available | ✅ Yes (e.g., every 2s) |
| Exception handling | ❌ Default only  | ✅ You choose to ignore |
| Use case           | Most scenarios  | Unpredictable delays   |


---------

#### **✅ What is Script Timeout?**

Script Timeout is used when **executing JavaScript code** via Selenium’s `JavascriptExecutor`.

It tells the driver:

> “⏳ Wait this long for any JavaScript to complete execution… or just give up!”

#### **🔍 Real-Life Analogy:**

> You ask your friend to do a task, and say: “If you don’t finish it in 10 seconds, I’m out.” 😤

> That’s what `scriptTimeout` does for browser scripts.

#### **🎯 When Do You Need Script Timeout?**

- You’re using `JavascriptExecutor` for actions like:

    - Triggering async JS events

    - Working with client-side validations

    - Running custom animations or heavy scripts

- Your JS might take some time to complete

- You want to **avoid getting stuck forever**



#### **🧪 Code Example – Script Timeout**

In [ ]:
package seleniumDemo1;

import java.time.Duration;
import org.openqa.selenium.JavascriptExecutor;
import org.openqa.selenium.WebDriver;
import org.openqa.selenium.chrome.ChromeDriver;

public class ScriptTimeoutDemo {
    public static void main(String[] args) {
        WebDriver driver = new ChromeDriver();

        // ⏱️ Set script timeout (max wait for JS to complete)
        driver.manage().timeouts().scriptTimeout(Duration.ofSeconds(10));

        driver.get("https://the-internet.herokuapp.com/");

        // ✅ JavaScriptExecutor for a dummy script
        JavascriptExecutor js = (JavascriptExecutor) driver;

        // Running a small JS that simulates async delay
        String script = "return new Promise(resolve => setTimeout(() => resolve('✅ JS done!'), 3000));";
        Object result = js.executeAsyncScript(script);

        System.out.println("Result from JavaScript: " + result);

        driver.quit();
    }
}


**⚠️ Note:**

- Script Timeout only applies to **async scripts**

- Regular `executeScript()` will NOT trigger this timeout

#### **✅ Summary of All Waits:**

| Type              | When to Use                               |
| ----------------- | ----------------------------------------- |
| `ImplicitWait`    | When elements take a bit of time globally |
| `ExplicitWait`    | For specific elements or conditions       |
| `FluentWait`      | For uncertain delays and retrying logic   |
| `PageLoadTimeout` | For full page to load                     |
| `ScriptTimeout`   | When executing **async JavaScript**       |
